# **Лабораторна робота №1**
## з дисципліни **"Методи оптимізації та дослідження операцій"**

##### *За темою:* **"Метод проекції градієнта"**

<div style="display: flex;">

<div style="flex: 1;">
    <i>Виконавці роботи:</i><br>
    бригада №21
    <br><br>
    <i><b>
    Баштовий Іван<br>
    Гавлицький Іван<br>
    Харитонов Олександр<br>
    Ходаковський Артур
    </i></b>
</div>

<div style="flex: 1;">
<i>Прийняли:</i>
<br><br><br><br>
<i><b>Яковлєва А. П.</i></b><br>
<i><b>Спекторський І. Я.</i></b>
</div>

</div>

In [1]:
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Початкові данні

In [2]:
# Задання символьних змінних для багатовимірних функцій
def create_symbols(n):
    return sp.symbols(f'x0:{n}')

x = create_symbols(2)

# Цільова функція
func = x[0]


In [3]:
# Генерація градієнта
def grad(func, symbols, values):
    grad_f = [sp.diff(func, sym) for sym in symbols]
    grad_f = [g.subs(dict(zip(symbols, values))) for g in grad_f]
    return np.array(grad_f, dtype=float)

# Генерація матриці Гессе
def hessian(func, symbols, values):
    hess_f = [[sp.diff(sp.diff(func, sym1), sym2) for sym1 in symbols] for sym2 in symbols]
    hess_f = [[h.subs(dict(zip(symbols, values))) for h in row] for row in hess_f]
    return np.array(hess_f, dtype=float)

### Реалізуємо основну функцію проекції точки на множину

In [4]:
def projection_function(point, params, constraint_type, params2=None):
    point = np.array(point).astype(float)
    params = np.array(params).astype(float)
    params2 = np.array(params2).astype(float)
    
    if constraint_type == 'Куля':
        if np.linalg.norm(point - params[:-1]) == 0:
            return point - params[:-1]
        
        return params[:-1] + (point - params[:-1])/(np.linalg.norm(point - params[:-1])) * params[-1]
    
    elif constraint_type == 'Еліпсоїд':
        dim = len(point)
        scaling_factors = np.sqrt(params[:-1])
        point_ = scaling_factors*point
        proj_ = (0,)*dim + (point_ - (0,)*dim)/(np.linalg.norm(point_ - (0,)*dim) + 1e-5) * params[-1]
        proj = proj_ * (1/scaling_factors)
        return proj
    
    elif constraint_type == 'Паралелепіпед':
        return [
            max(min(a_i, p2_i), p1_i)
            for a_i, p1_i, p2_i in zip(point, params, params2)
        ]
    
    elif constraint_type == 'Ортант':   
        return [
            max(a_i, p_i)
            for a_i, p_i in zip(point, params)
        ]
    
    elif constraint_type == 'Гіперплощина':
        return point + (params[-1] - params[:-1].dot(point))*params[:-1]/np.linalg.norm(params[:-1]) ** 2
    
    elif constraint_type == 'Півпростір':
        return point + max(0, params[-1] - params[:-1].dot(point))*params[:-1]/np.linalg.norm(params[:-1]) ** 2
    
    else:
        # Ітераційне наближення для довільних множин
        def objective(x):
            return np.linalg.norm(x - point)
        
        def constraint_eq(x):
            return eval(constraint_type)
        
        from scipy.optimize import minimize

        # Початкове наближення
        x0 = point
        
        # Оптимізація
        res = minimize(objective, x0, constraints={'type': 'eq', 'fun': constraint_eq})
        return res.x

Окремо прописуємо проекцію під нашу задачу

In [11]:
def task_projection(point):
    pr1 = projection_function(point, params=[1, 0], constraint_type='Ортант')
    return projection_function(pr1, [], '(x[0]-1) ** 3 - x[1]')

Метод золотого перетину та метод дроблення кроку

In [6]:
def golden_section_search_(function, left_boundary: float, right_boundary: float, epsilon: float) -> float:
    phi = (1 + np.sqrt(5))/2
    x1 = right_boundary - (right_boundary - left_boundary)/phi
    x2 = left_boundary + (right_boundary - left_boundary)/phi

    while abs(right_boundary - left_boundary) >= epsilon:
        if function(x1) >= function(x2):
            left_boundary = x1
        else:
            right_boundary = x2
        x1 = right_boundary - (right_boundary - left_boundary)/phi
        x2 = left_boundary + (right_boundary - left_boundary)/phi

    return (left_boundary + right_boundary)/2

In [7]:
def golden_section_search(function, left_boundary: float, right_boundary: float, epsilon: float) -> float:
    candidates = []
    intervals_no = 100
    step = (right_boundary - left_boundary)/intervals_no
    for i in np.arange(intervals_no):
        candidates.append(golden_section_search_(function, left_boundary + i*step, left_boundary + (i + 1)*step, epsilon=epsilon))
    candidates = np.array(candidates)
    min_index = np.argmin(np.vectorize(function)(candidates))
    return candidates[min_index]

In [8]:
def step_splitting(function, symbols, point: tuple, lmbda = 0.0005) -> float:
    def function_(x):
        return function.subs(dict(zip(symbols, x)))

    alpha = 1
    h = np.array(grad(function, symbols, point))
    while function_(point - alpha*h) > function_(point):
        # print(alpha)
        alpha = alpha*lmbda
    return alpha

Основний метод:

In [14]:
def gradient_projection_method(function, symbols, initial_point: tuple, task_projection_, epsilon=1e-5, type="minimization"):
    iterations = []
    alphas = []
    points = []
    function_values = []

    def function_(x):
        return function.subs(dict(zip(symbols, x)))

    if type == "step_splitting":
        def get_x_k(point: tuple):
            x_k_old = np.array(point)
            h = np.array(grad(function, symbols, x_k_old))
            alpha = step_splitting(function, symbols, x_k_old)
            x_k = x_k_old - alpha*h
            alphas.append(alpha)
            return task_projection_(x_k)

    else:
        def get_x_k(point: tuple):
            x_k_old = np.array(point)
            h = np.array(grad(function, symbols, x_k_old))
            alpha = golden_section_search(lambda a: function_(x_k_old - a*h), 0, 1, epsilon)
            x_k = x_k_old - alpha*h
            alphas.append(alpha)
            return task_projection_(x_k)
        

    x_k_old = task_projection_(initial_point)
    x_k = get_x_k(x_k_old)
        
    iteration = 1
    iterations.append(iteration)
    points.append(x_k)
    function_values.append(function_(x_k))
    print(f"Iteration: {iteration} | Point: {[x_k[i] for i in range(len(x_k))]} | Function value: {function_(x_k)}")    

    while (abs(function_(x_k) - function_(x_k_old)) >= epsilon) and (np.linalg.norm(x_k - x_k_old) >= epsilon):
        x_k_old = np.copy(x_k)
        x_k = get_x_k(x_k_old)

        iteration += 1
        iterations.append(iteration)
        points.append(x_k)
        function_values.append(function_(x_k))
        print(f"Iteration: {iteration} | Point: {[x_k[i] for i in range(len(x_k))]} | Function value: {function_(x_k)}")

    history = pd.DataFrame({"alpha": alphas}, index=iterations)
    history[["x_" + str(i+1) for i in range(len(x_k))]] = pd.DataFrame(np.array(points).tolist(), index=iterations)
    history["function_value"] = pd.DataFrame(function_values, index=iterations)
    return history

Основний цикл програми

In [15]:
gradient_projection_method(func, symbols=x, initial_point=[5, 5], task_projection_=task_projection)

Iteration: 1 | Point: [2.7256898240731453, 5.139113546266712] | Function value: 2.72568982407315
Iteration: 2 | Point: [2.7130442131704, 5.026963313838997] | Function value: 2.71304421317040
Iteration: 3 | Point: [2.700014762801076, 4.9131279930127665] | Function value: 2.70001476280108
Iteration: 4 | Point: [2.6865747505087834, 4.797519876844108] | Function value: 2.68657475050878
Iteration: 5 | Point: [2.6726932435956727, 4.680032402210459] | Function value: 2.67269324359567
Iteration: 6 | Point: [2.6583372836987973, 4.560564420119266] | Function value: 2.65833728369880
Iteration: 7 | Point: [2.643468268430613, 4.438987982262648] | Function value: 2.64346826843061
Iteration: 8 | Point: [2.6280429839904684, 4.315166910196786] | Function value: 2.62804298399047
Iteration: 9 | Point: [2.6120122859594264, 4.188948705544596] | Function value: 2.61201228595943
Iteration: 10 | Point: [2.595357778350202, 4.060450948539866] | Function value: 2.59535777835020
Iteration: 11 | Point: [2.57794007

,alpha,x_1,x_2,function_value
1,0.999996,2.725690,5.139114e+00,2.72568982407315
2,0.999996,2.713044,5.026963e+00,2.71304421317040
3,0.999996,2.700015,4.913128e+00,2.70001476280108
4,0.999996,2.686575,4.797520e+00,2.68657475050878
5,0.999996,2.672693,4.680032e+00,2.67269324359567
6,0.999996,2.658337,4.560564e+00,2.65833728369880
7,0.999996,2.643468,4.438988e+00,2.64346826843061
8,0.999996,2.628043,4.315167e+00,2.62804298399047
9,0.999996,2.612012,4.188949e+00,2.61201228595943
10,0.999996,2.595358,4.060451e+00,2.59535777835020


Випробування на різних початкових наближеннях

In [17]:
gradient_projection_method(func, symbols=x, initial_point=[120000000000000000000000, 20], type="step_splitting", task_projection_=task_projection)

Iteration: 1 | Point: [26241.418556674944, 19.0] | Function value: 26241.4185566749
Iteration: 2 | Point: [7.245610528926154, 243.6265950316002] | Function value: 7.24561052892615
Iteration: 3 | Point: [7.245542580778639, 243.61864306604963] | Function value: 7.24554258077864
Iteration: 4 | Point: [7.245474624678955, 243.61069090253665] | Function value: 7.24547462467896
Iteration: 5 | Point: [7.245406665285395, 243.60273852599389] | Function value: 7.24540666528539
Iteration: 6 | Point: [7.245338702506342, 243.59478592705133] | Function value: 7.24533870250634
Iteration: 7 | Point: [7.245270736471086, 243.58683311953405] | Function value: 7.24527073647109
Iteration: 8 | Point: [7.245202767173838, 243.57888010378235] | Function value: 7.24520276717384
Iteration: 9 | Point: [7.2451347945092754, 243.57092686716527] | Function value: 7.24513479450928
Iteration: 10 | Point: [7.245066818466175, 243.56297340806194] | Function value: 7.24506681846618
Iteration: 11 | Point: [7.244998839079888,

,alpha,x_1,x_2,function_value
1,1,26241.418557,1.900000e+01,26241.4185566749
2,1,7.245611,2.436266e+02,7.24561052892615
3,1,7.245543,2.436186e+02,7.24554258077864
4,1,7.245475,2.436107e+02,7.24547462467896
5,1,7.245407,2.436027e+02,7.24540666528539
...,...,...,...,...
17604,1,2.055380,1.175509e+00,2.05537956116730
17605,1,1.789248,4.916325e-01,1.78924804833023
17606,1,0.999929,-1.545438e-10,0.999928855307808
17607,1,0.999999,-1.391130e-22,0.999999373490764


In [19]:
gradient_projection_method(func, symbols=x, initial_point=[-1000000000000000, -1000000000000000], task_projection_=task_projection)

Iteration: 1 | Point: [0.9999993734907638, -1.3911299582599802e-22] | Function value: 0.999999373490764


,alpha,x_1,x_2,function_value
1,0.999996,0.999999,-1.391130e-22,0.999999373490764


### Ярна функція

In [26]:
x_temp = create_symbols(2)
func_temp = 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

def yard_projection(point):
    return projection_function(point, [0, 0, 1], 'Куля')

In [27]:
gradient_projection_method(func_temp, symbols=x_temp, initial_point=[1, 1], type="step_splitting",task_projection_=yard_projection)

Iteration: 1 | Point: [0.7316397095006484, 0.6816915251650167] | Function value: 2.21516276789719
Iteration: 2 | Point: [0.7486783752496119, 0.6629333981883861] | Function value: 1.11202711389440
Iteration: 3 | Point: [0.7604313998138561, 0.649418267511116] | Function value: 0.563801172440383
Iteration: 4 | Point: [0.7685197731905017, 0.6398260374626997] | Function value: 0.295680510027568
Iteration: 5 | Point: [0.7740844949719884, 0.6330822968966686] | Function value: 0.165792708123993
Iteration: 6 | Point: [0.7779147201850742, 0.6283698656996354] | Function value: 0.103231995655114
Iteration: 7 | Point: [0.7805527854549726, 0.6250898728330857] | Function value: 0.0732071754419945
Iteration: 8 | Point: [0.782370855064849, 0.6228128492132261] | Function value: 0.0588300582112031
Iteration: 9 | Point: [0.7836244337959006, 0.6212348563611462] | Function value: 0.0519558390975816
Iteration: 10 | Point: [0.7844891196692425, 0.620142581283189] | Function value: 0.0486722154458052
Iteration:

,alpha,x_1,x_2,function_value
1,0.0005,0.731640,0.681692,2.21516276789719
2,0.0005,0.748678,0.662933,1.11202711389440
3,0.0005,0.760431,0.649418,0.563801172440383
4,0.0005,0.768520,0.639826,0.295680510027568
5,0.0005,0.774084,0.633082,0.165792708123993
6,0.0005,0.777915,0.628370,0.103231995655114
7,0.0005,0.780553,0.625090,0.0732071754419945
8,0.0005,0.782371,0.622813,0.0588300582112031
9,0.0005,0.783624,0.621235,0.0519558390975816
10,0.0005,0.784489,0.620143,0.0486722154458052
